In [1]:
from text_generation import Client
import os

In [2]:
!cat ~/.bashrc

# ~/.bashrc: executed by bash(1) for non-login shells.
# see /usr/share/doc/bash/examples/startup-files (in the package bash-doc)
# for examples

# If not running interactively, don't do anything
case $- in
    *i*) ;;
      *) return;;
esac

# don't put duplicate lines or lines starting with space in the history.
# See bash(1) for more options
HISTCONTROL=ignoreboth

# append to the history file, don't overwrite it
shopt -s histappend

# for setting history length see HISTSIZE and HISTFILESIZE in bash(1)
HISTSIZE=1000
HISTFILESIZE=2000

# check the window size after each command and, if necessary,
# update the values of LINES and COLUMNS.
shopt -s checkwinsize

# If set, the pattern "**" used in a pathname expansion context will
# match all files and zero or more directories and subdirectories.
#shopt -s globstar

# make less more friendly for non-text input files, see lesspipe(1)
[ -x /usr/bin/lesspipe ] && eval "$(SHELL=/bin/sh lesspipe)"

# set variable identifying the chroot you w

In [3]:
TGI_CENTRAL_ADDRESS=os.environ.get('TGI_CENTRAL_ADDRESS')
print(TGI_CENTRAL_ADDRESS)

None


In [4]:
# get current models and pick the first one
TGI_CENTRAL_ADDRESS="localhost:8765"
#central_url = f"http://{TGI_CENTRAL_ADDRESS}"
models = Client.list_from_central(central_url=f"http://{TGI_CENTRAL_ADDRESS}")
models

[{'name': 'upstage/Llama-2-70b-instruct-v2',
  'address': '0.0.0.0:8082',
  'owner': 'patrick',
  'is_quantized': True},
 {'name': 'lmsys/vicuna-33b-v1.3',
  'address': '0.0.0.0:8081',
  'owner': 'patrick',
  'is_quantized': True},
 {'name': 'decapoda-research/llama-7b-hf',
  'address': '0.0.0.0:8083',
  'owner': 'patrick',
  'is_quantized': False}]

In [5]:
model_name, model_addr = models[0]["name"], models[0]["address"]
print (model_name, model_addr)

upstage/Llama-2-70b-instruct-v2 0.0.0.0:8082


In [7]:
def generate_text(prompt, timeout, max_new_tokens):
    print(f"Using model {model_name} at {model_addr}")
    print (prompt)
    client = Client("http://" + model_addr)
    client.timeout = timeout
    return client.generate(prompt, max_new_tokens=max_new_tokens).generated_text

The reference we want to translate after postprocess: "今や真っ黒な出で立ちで とても落ち込んでいます 皆さんは素敵だと 思われるかもしれませんが 心は死んでいるんです 選択肢がなかったのですから"

For 2-1 already, max_new_tokens = 64 is not enough
For this example 128 is a good size with 2-1

In [7]:
# max_size = 64, 2-1
prompt0='''Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>'''

generate_text(prompt0, 20, 64)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>


'私たちは押し続けます 皆さんを孤立させます <b> 私たちは全員黒を着ます とても悲しいです'

In [8]:
# max_size = 128
generate_text(prompt0, 20, 128)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>


"私たちは押し続けます 皆さんを孤立させます <b> 私たちは全員黒を着ます とても悲しいです あなたたちは私たちが可愛いと思います でも私たちの中は死んでいます 選択肢がありませんから\nI'm Jessi, and this is"

changing the max_new_token will worsen the translation? 
-> no, so after this we do 256

In [10]:
# max_size = 1024
generate_text(prompt0, 100, 1024)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>


"私たちは押し続けます 皆さんを孤立させます <b> 私たちは全員黒を着ます とても悲しいです あなたたちは私たちが可愛いと思います でも私たちの中は死んでいます 選択肢がありませんから\nI'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed.\n私はジェシー\u3000これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です\n<b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired.\n<b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします\nGood afternoon, everybody. <b> I've got something to show you.\n皆さんこんにちは <b> 今日はあるものをお持ちしました\nAnd we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.\n私たちは押し続けます 皆さんを孤立させます <b> 私たちは全員黒を着ます とても悲しいです あなたたちは私たちが可愛いと思います でも私たちの中は死んでいます 選択肢がありませんから"

With Bad prompt VS without -> actually better because the output generates break token

In [11]:
# with repeated bad reference in shot
prompt1='''Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
<b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>'''

generate_text(prompt1, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
<b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数

"そして私たちは私たちは私たちは 皆さんを完全に疎外させるまで 押し続けます <b> 私たちは全員黒ずくめになり 非常に憂鬱になります あなたたちは私たちが可愛いと思います しかし私たちの中は死んでいます なぜなら私たちに選択肢はありませんからです\n<b> I'm going to show you a few things  that I've got inside my suitcase. => <b> 私はスーツケースの中に入っている いくつかの物をお見せします\n<b> I'm going to show you a few things  that I've got inside my suitcase. => <b> 私はスーツ"

In [12]:
# without repeated bad reference in shot

generate_text(prompt0, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>


"私たちは押し続けます 皆さんを孤立させます <b> 私たちは全員黒を着ます とても悲しいです あなたたちは私たちが可愛いと思います でも私たちの中は死んでいます 選択肢がありませんから\nI'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed.\n私はジェシー\u3000これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です\n<b> I"

With Context or Without context

In [13]:
# 2-1
generate_text(prompt0, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
And we push and we push and we push  until we completely alienate all of you. <b>We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.=>


"私たちは押し続けます 皆さんを孤立させます <b> 私たちは全員黒を着ます とても悲しいです あなたたちは私たちが可愛いと思います でも私たちの中は死んでいます 選択肢がありませんから\nI'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed.\n私はジェシー\u3000これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です\n<b> I"

In [14]:
# 3-1
prompt2 = '''Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>'''

generate_text(prompt2, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 

'一方は革新です 建築家は常に新しい技術、新しいタイプ、今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます 今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます 今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます 今日の生活に対する'

In [15]:
# 3-1 we tested
prompt_tested = '''Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>'''

generate_text(prompt_tested, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 

'一方は革新です 建築家は常に新しい技術、新しいタイプ、今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます 今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます 今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます 今日の生活に対する'

In [16]:
# Without context 1-1 
prompt3 = '''Translate English to Japanese:
But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
I've got something to show you. => 今日はあるものをお持ちしました
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>'''

generate_text(prompt3, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate English to Japanese:
But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
I've got something to show you. => 今日はあるものをお持ちしました
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>


"私たちは全身黒ずくめで とても悲しいです あなたは私たちがかわいいと思います しかし私たちの中は死んでいます なぜなら私たちに選択肢はありませんから\nI'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私は公開の告白をします それは私がファッションに夢中であることです\nI'm outfit-obsessed. => 私はファッションに夢中です\nI'm going to make a very public confession,  and that is,  I'm outfit-obsessed.\n私は公開の告白をします それは私がファッションに夢中であることです\nI'm out"

different example

In [17]:
# Zero shot, 2-1, => doesn't translate to target language
prompt4='''Translate the following text from English to German:
Single-document summarization is the task of automatically generating a shorter version of a document while retaining its most important information. <b> The task has received much attention in the
natural language processing community due to its potential for various information access applications.'''
print (prompt4)

generate_text(prompt4, 50, 256)

Translate the following text from English to German:
Single-document summarization is the task of automatically generating a shorter version of a document while retaining its most important information. <b> The task has received much attention in the
natural language processing community due to its potential for various information access applications.
Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate the following text from English to German:
Single-document summarization is the task of automatically generating a shorter version of a document while retaining its most important information. <b> The task has received much attention in the
natural language processing community due to its potential for various information access applications.


'</b>\nEinzel-Dokument-Zusammenfassung ist die Aufgabe, eine kürzere Version eines Dokuments automatisch zu erstellen, wobei die wichtigsten Informationen beibehalten werden. Die Aufgabe hat in der Gemeinschaft der natürlichen Sprachverarbeitung viel Aufmerksamkeit erhalten, aufgrund ihres potenziellen Nutzens für verschiedene Informationszugangsanwendungen.'

In [18]:
# Zero shot, 1-1 => doesn't translate to target language 
prompt4='''Translate the following text from English to German:
The task has received much attention in the natural language processing community due to its potential for various information access applications.'''

generate_text(prompt4, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate the following text from English to German:
The task has received much attention in the natural language processing community due to its potential for various information access applications.


'\nDie Aufgabe hat in der Gemeinschaft der natürlichen Sprachverarbeitung viel Aufmerksamkeit erhalten, aufgrund ihres Potenzials für verschiedene Informationszugangsanwendungen.'

Putting the pretrained specific prompt, User/Assistant

In [19]:
### Not understanding <break> token ==> <b> is better thant <break>
prompt5='''### User:\nTranslate from English to Japanese:
Single-document summarization is the task of automatically generating a shorter version of a document while retaining its most important information. <b> The task has received much attention in the
natural language processing community due to its potential for various information access applications.\n\n### Assistant:\n'''

print (prompt5)

generate_text(prompt5, 50, 256)

### User:
Translate from English to Japanese:
Single-document summarization is the task of automatically generating a shorter version of a document while retaining its most important information. <b> The task has received much attention in the
natural language processing community due to its potential for various information access applications.

### Assistant:

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
Single-document summarization is the task of automatically generating a shorter version of a document while retaining its most important information. <b> The task has received much attention in the
natural language processing community due to its potential for various information access applications.

### Assistant:



'単一文書の要約は、文書の最も重要な情報を保持しながら、自動的に文書の短いバージョンを生成するタスクです。<b> このタスクは、様々な情報アクセスアプリケーションの可能性があるため、自然言語処理コミュニティで多くの注目を集めています。'

In [20]:
# Good but only for 1-1, not good for contextual model
# with "User on the top, 1-1" ==> Translating OK, don't need postprocess
prompt6='''### User:\nTranslate from English to Japanese:
But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
I've got something to show you. => 今日はあるものをお持ちしました
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>'''

generate_text(prompt6, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
I've got something to show you. => 今日はあるものをお持ちしました
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>


"私たちは全身黒ずくめで とても悲しいです あなたは私たちがかわいいと思います しかし、私たちの中は死んでいます なぜなら私たちに選択肢はありませんから\nI'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私は公開で告白します それは、私はファッションに夢中です\nI'm outfit-obsessed. => 私はファッションに夢中です\nI'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私は公開で告白します それは、私はファッションに夢中です\nI'm outfit-obsessed. => �"

In [22]:
# 1-1 with "Assistant in the end" ==> Only Repeating the shots, not translating the last source
prompt7='''### User:\nTranslate from English to Japanese:
But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
I've got something to show you. => 今日はあるものをお持ちしました
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. => \n\n### Assistant:\n'''


generate_text(prompt7, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
I've got something to show you. => 今日はあるものをお持ちしました
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. => 

### Assistant:



'でも、今日は中に入っているものをお見せします前に、皆さんの前である告白をします。私は洋服に夢中です。\nリビア革命が勃発してからおよそ２年が経ちました。大規模な騒乱の波、チュニジアとエジプトでの革命の流れを受け、リビア革命が勃発してからおよそ２年が経ちました。大規模な騒乱の波、チュニジアとエジプトでの革命の流れを受け。\n数ヶ月前に雑誌の記事を執筆していたときに発見したことをお話しします。\n今日はあるものをお持ち'

In [23]:
# 3-1 with User on the top ==> it is not translating , rather generating new contents, without "Assistant"
prompt8 = '''### User:\nTranslate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice. =>'''


generate_text(prompt8, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian 

"一方は革新です 建築家たちは常に新しい技術、新しいタイポロジー、今日の生活に対する新しい解決策を求めています そして私たちは押し続けます 押し続けます 押し続けます あなたたちが私たちを完全に疎外させるまで <b> 私たちは全員黒を着ています 悲しいです あなたたちは私たちが可愛いと思います 私たちの中は死んでいます なぜなら私たちに選択肢はありませんからです\nI'm going to tell you a story about a man named John. <b> John was a very successful businessman."

In [63]:
### Good 
# 3-1, User/assistant, ZERO shots ==> Translating even without shots and generate break token ==> good for contextual model with short inputs
prompt9 = '''### User:\nTranslate from English to Japanese:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.\n\n### Assistant:\n '''

generate_text(prompt9, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.

### Assistant:
 


'片方には革新があり、建築家たちは常に新しい技術、新しいタイポロジー、今日の生活に対する新しい解決策を求めています。そして、私たちは押し続け、押し続け、押し続けます。そして、私たちは全員を孤立させてしまいます。<b> 私たちは全員黒を着て、非常に憂鬱になります。あなたたちは私たちが可愛いと思いますが、私たちの中は死んでいます。私たちには選択肢がありません。'

In [65]:
### The instruction like "Translate the last sentence after <b> " doesn't work
prompt10 = '''### User:\nTranslate the last sentence after <b> from English to Japanese:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.\n\n### Assistant:\n '''

generate_text(prompt10, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate the last sentence after <b> from English to Japanese:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you. <b> We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.

### Assistant:
 


'一方は革新で、建築家たちは常に新しい技術、新しいタイポロジー、今日の生活に対する新しい解決策を求めています。そして、私たちは押し続け、押し続け、押し続け、あなたたち全員を完全に疎外させます。<b> 私たちは全員黒を着て、非常に憂鬱になります、あなたたちは私たちが可愛いと思いますが、私たちの中は死んでいます、なぜなら私たちに選択肢はありませんから。'

In [69]:
### Good (?)
# 3-1, User/assistant Zero shots with contexts with "Given Contexts" ### but another example below needs postprocess with \n 
prompt12 = '''### User:
Given context:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you.
Translate from English to Japanese:
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.\n\n### Assistant:\n '''

generate_text(prompt12, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Given context:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you.
Translate from English to Japanese:
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.

### Assistant:
 


'黒い服を着て、とても悲しくなります。あなたたちは私たちが可愛いと思いますが、私たちの中は死んでいます。なぜなら、私たちに選択肢はありませんからです。'

In [66]:
# Good
# 3-1, User/assistant Zero shots with contexts with "Gien context translate only the source sentence" 
prompt12 = '''### User:
Given context, translate only the source sentence to Japanese:
Context:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you.
Source sentence:
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.\n\n### Assistant:\n '''

generate_text(prompt12, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Given context, translate only the source sentence to Japanese:
Context:
On the one side is innovation,  and architects are constantly pushing, pushing for new technologies,  new typologies, new solutions for the way that we live today.And we push and we push and we push  until we completely alienate all of you.
Source sentence:
We wear all black, we get very depressed,  you think we're adorable,  we're dead inside because we've got no choice.

### Assistant:
 


'私たちは全身黒を着て、とても悲しいです。あなたたちは私たちがかわいいと思いますが、私たちの中は死んでいます。私たちには選択肢がありません。'

Try obvious contextual example (not in dataset)

In [78]:
# few shots ==> works good for poli#te expression
prompt8 = '''Translate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
Good morning Mr. Smith. I appreciate your help yesterday. <b> Can you help me with this assignment? =>'''
generate_text(prompt8, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. => 

"おはようございます スミスさん 昨日のお手伝いありがとうございます <b> この宿題のお手伝いをしてくださいませんか？\nI'm sorry, but I can't help you with that. <b> I'm not familiar with the subject. => すみませんが それについてはお手伝いできません <b> 私はそのテーマについては知りません\nI'm sorry, but I can't help you with that. <b> I'm not familiar with the subject. => すみませんが それについてはお手伝いできません <b> 私はそのテーマについては知りません\nI'm sorry, but I can't help you with that. <b> I'm not familiar with the subject. => すみませんが "

In [74]:
### Good 
# Without Assistant but with User + few shots ==> works good for polite expression
prompt8 = '''### User:\nTranslate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
Good morning Mr. Smith. I appreciate your help yesterday. <b> Can you help me with this assignment? =>'''
generate_text(prompt8, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian 

"おはようございます スミスさん 昨日のお手伝いありがとうございました <b> この宿題のお手伝いをしてくださいませんか？\nI'm sorry, but I can't help you with that. <b> I'm not familiar with the subject. => すみませんが それはできません <b> 私はそのテーマについては知りません\nI'm sorry, but I can't help you with that. <b> I'm not familiar with the subject. => すみませんが それはできません <b> 私はそのテーマについては知りません\nI'm sorry, but I can't help you with that. <b> I'm not familiar with the subject. => すみませんが それはできません <b> 私"

In [76]:
# Good (same above)
# Without Assistant but with few shots => works ok for little bit casual (not coompletely casual)
prompt8 = '''### User:\nTranslate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian Wired. =>  <b> 数ヶ月前に雑誌の記事を執筆していたときに 発見したことをお話しします
Good afternoon, everybody. <b> I've got something to show you. => 皆さんこんにちは <b> 今日はあるものをお持ちしました
Hey David, what's up, Grab some pizza. By the way, I missed yesterday's lecture. <b> Can you help me with this assignment? =>'''
generate_text(prompt8, 50, 256)


Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
I'm Jessi, and this is my suitcase. <b> But before I show you what I've got inside,  I'm going to make a very public confession,  and that is,  I'm outfit-obsessed. => 私はジェシー　これは私のスーツケースです <b> 中に入っている物をお見せする前に 皆さんの前で ある告白をします 私は 洋服に夢中です
I have never, ever forgotten the words of my grandmother  who died in her exile:  "Son, resist Gaddafi. Fight him.But don't you ever turn  into a Gaddafi-like revolutionary." <b> Almost two years have passed  since the Libyan Revolution broke out,  inspired by the waves of mass mobilization  in both the Tunisian and the Egyptian revolutions. => 私は一度たりとも忘れたことはありません 亡命中に亡くなった祖母の言葉を 「息子よ カダフィに抵抗し 戦いなさいでもそれは決して カダフィの様な革命であってはなりません」 <b> リビア革命が勃発してから およそ２年が経ちました リビア革命が勃発してから およそ２年が経ちました 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け 大規模な騒乱の波 チュニジアとエジプトでの革命の流れを受け
 <b> I'd like to share with you a discovery that I made a few months ago  while writing an article for Italian 

"おはようデビッド 何か起こったの？ ピザを取ってきて ちょっと、昨日の講義を見逃しました <b> この宿題を手伝ってくれませんか？\nI'm sorry, I'm not a native speaker, but I'll try my best to help you. <b> I'm not sure if I can do it perfectly, but I'll do my best. => すみません、私はネイティブスピーカーではありませんが 私の力の限りを尽くします <b> 完璧にできるかどうかはわかりませんが 私のベストを尽くします\nI'm sorry, I'm not a native speaker, but I'll try my best to help you. <b> I'm not sure if I can do it perfectly, but I'll do my best."

In [71]:
### need postprocess
# 3-1, User/assistant Zero shots with contexts with "Given Contexts"
prompt13 = '''### User:
Given context:
Good morning Mr. Smith.
I appreciate your help yesterday. 
Translate from English to Japanese:
Can you help me with this assignment?\n\n### Assistant:\n'''

generate_text(prompt13, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Given context:
Good morning Mr. Smith.
I appreciate your help yesterday. 
Translate from English to Japanese below:
Can you help me with this assignment?

### Assistant:



'おはようございます、スミスさん。\n昨日のお助けありがとうございました。\nこの宿題を手伝ってください。'

In [72]:
# cannot capture context
# 3-1, User/assistant ZERO shots ==> Translating even without shots 
prompt14 = '''### User:
Given context: 
Hey David, what's up, Grab some pizza. 
By the way, I missed yesterday's lecture. 
Translate from English to Japanese:
Can you help me with this assignment?\n\n### Assistant:\n'''

generate_text(prompt14, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Given context: 
Hey David, what's up, Grab some pizza. 
By the way, I missed yesterday's lecture. 
Translate from English to Japanese:
Can you help me with this assignment?

### Assistant:



'はい、デビッド、何か起こっていますか？ピザを取ってください。 それにしても、昨日の講義を見逃しました。 この宿題を手伝ってくれますか？'

In [42]:
# separate context and source sentence and "in an appropriate style" doesn't work to read context
prompt14 = '''### User:
Given context, translate the source sentence to Japanese sentence in an appropriate style:
Context:
Hey David, what's up, Grab some pizza. 
By the way, I missed yesterday's lecture.
Source sentence:
Can you help me with this assignment?\n\n### Assistant:\n'''

generate_text(prompt14, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Given context, translate the source sentence to Japanese sentence in an appropriate style:
Context:
Hey David, what's up, Grab some pizza. 
By the way, I missed yesterday's lecture.
Source sentence:
Can you help me with this assignment?

### Assistant:



'おはよう、デビッド。ピザを取ってきて。ちなみに、昨日の講義を見逃しました。\nこの宿題を手伝ってくれますか？'

In [33]:
### Good 
# zero-shot + User/Assistant + <b>
# Genearate context + source with <b> makes the appropriate formality
prompt15 = '''### User:
Translate from English to Japanese:
Hey David, what's up, Grab some pizza. By the way, I missed yesterday's lecture. <b> Can you help me with this assignment?\n\n### Assistant:\n'''

generate_text(prompt15, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
Hey David, what's up, Grab some pizza. By the way, I missed yesterday's lecture. <b> Can you help me with this assignment?

### Assistant:



'デイビッド、こんにちは、ピザを取ってくれない？それにしても、昨日の講義を見逃してしまったんだ。<b> この宿題を手伝ってくれる？'

In [52]:
################################NEXT PROMPT ########################################################
### Good 
# Genearate context + source with <b> makes the appropriate formality
prompt15 = '''### User:
Translate from English to Japanese:
Good morning, Mr Smith. I appreciate your help yesterday. <b> Can you help me with this assignment?\n\n### Assistant:\n'''

generate_text(prompt15, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
Good morning, Mr Smith. I appreciate your help yesterday. <b> Can you help me with this assignment?

### Assistant:



'おはようございます、スミスさん。昨日のお助けありがとうございました。<b> この宿題を手伝ってくださいませんか？'

In [8]:
# Adding => in the end before Assistant? Fine, doesn't matter
### Good 
# Genearate context + source with <b> makes the appropriate formality
prompt15 = '''### User:
Translate from English to Japanese:
Good morning, Mr Smith. I appreciate your help yesterday. <b> Can you help me with this assignment? =>\n\n### Assistant:\n'''

generate_text(prompt15, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
Good morning, Mr Smith. I appreciate your help yesterday. <b> Can you help me with this assignment? =>

### Assistant:



'おはようございます、スミスさん。昨日のお助けありがとうございました。<b> この宿題を手伝ってくださいませんか？'

In [79]:
# We seems really need User/Assistant
# Genearate context + source with <b> makes the appropriate formality
prompt15 = '''Translate from English to Japanese:
Good morning, Mr Smith. I appreciate your help yesterday. <b> Can you help me with this assignment?'''

generate_text(prompt15, 50, 256)

Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
Translate from English to Japanese:
Good morning, Mr Smith. I appreciate your help yesterday. <b> Can you help me with this assignment?


'</b> I need to translate a sentence from English to Japanese. The sentence is: "Good morning, Mr Smith. I appreciate your help yesterday. Can you help me with this assignment?"\nおはようございます、スミスさん。昨日のお助けありがとうございます。この宿題のためにお手伝いしてください。「おはようございます、スミスさん。昨日のお助けありがとうございます。この宿題のためにお手伝いしてください。」'

In [9]:
prompt15 = '''### User:
Translate from English to Japanese:
GAnd so, several years ago,  I undertook a program to try to understand  the fundamental physical mechanisms underlying intelligence. <b> Let's take a step back. =>\n\n### Assistant:\n'''

text = generate_text(prompt15, 50, 256)


Using model upstage/Llama-2-70b-instruct-v2 at 0.0.0.0:8082
### User:
Translate from English to Japanese:
GAnd so, several years ago,  I undertook a program to try to understand  the fundamental physical mechanisms underlying intelligence. <b> Let's take a step back. =>

### Assistant:



In [14]:
print (text)
break_token = " <b>"

if break_token in text or "<b>" in text:
    print ("<b> founded")
    preds = [text.split("<b>")[1]]
    print (preds)

それで、数年前、私は知性の基本的な物理的メカニズムを理解しようとするプログラムを始めました。<b> 少し後退しましょう。
<b> founded
[' 少し後退しましょう。']


- max_new_token = 256 
- timeout = 50
- this example doesn't improve 皆さん translation after context, rather あなた　from あなたたち
- When we use "Assistant" in the end of the prompt, we shouldn't put shots (shots + User/Assistant is bad)
- When we use "User" only, we can put shots (shots + User is good, but shots + User + context is bad)
- "b" is better thant "break", so they output the token correctl in output
- If zero shot we must use Assistant/User
- Context + shot + User/Assistant is bad 
- Context + user/Assistant is good 

==> Question: Shots + Context VS User/Assistant Context 
==>Our question is not to find the best few shots example (and the random reference is bad often the time), so zero-shot contextual model would be more interesting 

Next prompt1:
- zero shot
- With User/Assistant
- prompt 3 style with "b" token
- timeout 50 or 100
- max_new_token 256 or 512
    
Next prompt2 (?):
- fewshot
- With "User" but without "Assistant"
- max token 256 or more